In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
test_data = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/test.csv')
train_data = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/train.csv')

In [ ]:
print(test_data.head(3))
print(train_data.head(3))

In [ ]:
print(train_data.isnull().sum())

*The data is free form null values*

# Checking correlations

In [ ]:
fig, ax = plt.subplots(figsize= (16.5, 15))
sns.heatmap(train_data.corr(), annot= True,  cmap="YlGnBu")

# Observations
* id is not a feature
* most of the features do not have a good correaltion with the target 
* some of the features are correlated to each other to higher degrees 

In [ ]:
highCorrFeatures = []
corrValue = []
otherFeatures = []
for feature in train_data.columns.values:
    for otherFeature in train_data.columns.values:
        if( abs(train_data[feature].corr(train_data[otherFeature])) >= 0.65 ):
            highCorrFeatures.append(feature)
            otherFeatures.append(otherFeature)
            corrValue.append(abs(train_data[feature].corr(train_data[otherFeature])))
    

In [ ]:
corrDF = pd.DataFrame(data = corrValue, index = highCorrFeatures, columns = ['value'])
corrDF['otherFeature'] = otherFeatures
corrDF = corrDF[ abs(corrDF['value'] -  1.0) >= 0.01] #comparing float values directly can cause unexpected results
print(corrDF)


In [ ]:
fig, ax = plt.subplots(figsize= (16.5, 15))
sns.barplot( corrDF['value'], corrDF.index.values)

We can see the features in the plot are correlated to 1 or more features and hence need to be taken care of

*cont2, cont3, cont4, cont5, cont8, cont14 seem to be reletavely less correlated to other features and can be used directly*

In [ ]:
drop_features = ['cont6', 'cont7', 'cont9', 'cont10', 'cont12']

In [ ]:
train_data.head(2)
X = train_data.drop( ['target', 'id'], axis  = 1)
y = train_data['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3)

In [ ]:
clf = GradientBoostingRegressor(n_estimators = 100, learning_rate = 0.1, max_depth = 1, random_state = 0, loss = 'ls')
clf.fit(X_train, y_train)

In [ ]:
score = 1 - mean_squared_error(y_test, clf.predict(X_test))
print(score)